# Examples

## Import this library

In [1]:
@file:DependsOn("io.github.stream29:regexdsl-jvm:0.1.0")
import io.github.stream29.regexdsl.*

## hello world

In [2]:
val helloWorldRegex = buildRegex {
    match("hello")
    match(spaceChar)
    match("world")
}
println(helloWorldRegex.pattern)
println(helloWorldRegex.matches("hello world"))

hello\sworld
true


## Phone number

In [3]:
val phoneNumberRegex = buildRegex {
    match(digitChar, Quantifier.exactly(3))
    match("-")
    match(digitChar, Quantifier.exactly(3))
    match("-")
    match(digitChar, Quantifier.exactly(4))
}
println(phoneNumberRegex.pattern)
println(phoneNumberRegex.matches("123-456-7890"))

\d{3}-\d{3}-\d{4}
true


## Email

In [4]:
val emailRegex = buildRegex {
    matchStringBegin()
    matchNamedGroup("username") {
        matchCharacterSet(Quantifier.atLeastOne) {
            add(digitChar)
            add(wordChar)
            add('_')
        }
    }
    match("@")
    matchNamedGroup("domain") {
        match(wordChar, Quantifier.atLeastOne)
        matchUncapturedGroup(Quantifier.atLeastOne) {
            match('.')
            match(wordChar, Quantifier.atLeastOne)
        }
    }
    matchStringEnd()
}

println(emailRegex.pattern)
val emailMatchGroups = emailRegex.matchEntire("stream@dify.ai")!!.groups
println(emailMatchGroups["username"])
println(emailMatchGroups["domain"])

^(?<username>[\d\w_]+)@(?<domain>\w+(?:\.\w+)+)$
MatchGroup(value=stream, range=0..5)
MatchGroup(value=dify.ai, range=7..13)


## URL

In [7]:
val urlRegex = buildRegex {
    matchStringBegin()
    matchNamedGroup("protocol") {
        match("http")
        match('s', Quantifier.atMostOne)
    }
    match("://")
    matchIndexedGroup {
        matchNamedGroup("ip") {
            match(digitChar, Quantifier.atLeastOne)
            match('.')
            match(digitChar, Quantifier.atLeastOne)
            match('.')
            match(digitChar, Quantifier.atLeastOne)
            match('.')
            match(digitChar, Quantifier.atLeastOne)
            matchIndexedGroup(Quantifier.atMostOne) {
                match(':')
                match(digitChar, Quantifier.anyTimes)
            }
        }
        insertOr()
        matchNamedGroup("domain") {
            matchIndexedGroup(Quantifier.anyTimes) {
                match(wordChar, Quantifier.atLeastOne)
                match('.')
            }
            match(wordChar, Quantifier.atLeastOne)
        }
    }
    matchNamedGroup("path") {
        matchIndexedGroup(Quantifier.anyTimes) {
            match('/')
            matchNegatedCharacterSet(Quantifier.atLeastOne) {
                add('/')
            }
        }
    }
    matchStringEnd()
}

println(urlRegex.pattern)
val urlMatchGroups = urlRegex.matchEntire("https://github.com/Stream29/RegexDsl")!!.groups
println(urlMatchGroups["protocol"])
println(urlMatchGroups["ip"])
println(urlMatchGroups["domain"])
println(urlMatchGroups["path"])

^(?<protocol>https?)://((?<ip>\d+\.\d+\.\d+\.\d+(:\d*)?)|(?<domain>(\w+\.)*\w+))(?<path>(/[^/]+)*)$
MatchGroup(value=https, range=0..4)
null
MatchGroup(value=github.com, range=8..17)
MatchGroup(value=/Stream29/RegexDsl, range=18..35)


## Groups and backreferences

In [8]:
val backReferenceRegex = buildRegex {
    val group = matchNamedGroup("repeatWord") {
        matchWordBoundary()
        match(wordChar, Quantifier.atLeastOne)
        matchWordBoundary()
    }
    match(anyChar, Quantifier.anyTimes)
    match(group)
}
println(backReferenceRegex.pattern)
val backReferenceMatchGroups = backReferenceRegex.matchEntire("Stream opened a FileStream")!!.groups
println(backReferenceMatchGroups["repeatWord"])

(?<repeatWord>\b\w+\b).*\k<repeatWord>
MatchGroup(value=Stream, range=0..5)
